Fine-Tuning Frontier Models
===========================

3 states to fine-tuning with OpenAI:

1. Create training dataset in jsonl format and upload to OpenAI
2. Run training - training loss and validation loss should decrease
3. Evaluate results, tweak and repeat

Key objectives of fine-tuning for Fronier models:

- Setting style or tone in a way that can't be acheived with prompting
- Improving the reliability of producing a type of output
- Correcting failures to follow complex prompts
- Handling edge cases
- Performing a new skill or task that's hard to articulate in a prompt

## Result

<img src="./../images/Product-Pricer-Fine-Tuned-Results-GPT-Mini-Test-1.jpg" alt="Distribution of Prices Predicted Using Fine Tuned GPT 4o Mini" />

Result: Does worse than untrained Frontier models but better than most traditional ML methods and improved in reducing large outliers.

- GPT Fine Tuned Pricer Error=$101.59
- RMSLE=0.80
- Hits=41.6%

## Compare Results to Basline Models

<img src="./../images/Product-Pricer-FIne-Tuned-Baseline-Traditional-ML-Models-Results.jpg" alt="Average Absolute Prediction Error Baseline Models vs FIne Tuned Frontier Model" />

A problem like ours doesn't benefit significantly from Fine Tuning

- The problem and style of output can be clearly specified in a prompt
- The model can take advantage of its enourmous world knowldge from its pre-training; providing a few hundred prices doesn't help

# Dependencies

In [ ]:
# imports

import os
import re
import math
import json
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from collections import Counter
from openai import OpenAI
from anthropic import Anthropic

# Setup

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# moved our Tester into a separate package
# call it with Tester.test(function_name, test_dataset)
import sys
sys.path.append('../testing/')
from items import Item
from testing import Tester

In [ ]:
openai = OpenAI()

In [ ]:
%matplotlib inline

# Load Curated Datasets

Load in the pickle files create during data curation.

In [ ]:
# Load in the pickle files:

with open('../data/large-datasets/train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('../data/large-datasets/test.pkl', 'rb') as file:
    test = pickle.load(file)

In [ ]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 200 examples (and 1 epoch)

fine_tune_train = train[:200]
fine_tune_validation = train[200:250]

# Fine-Tuning GTP

## Step 1: Preparing the Data

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI.

OpenAI expects data in JSONL format: rows of JSON each containing messages in the usual prompt format.

    {"messages" : [{"role": "system", "content": "You estimate prices..."}]}
    {"messages" : [{"role": "system", "content": "You estimate prices..."}]}
    {"messages" : [{"role": "system", "content": "You estimate prices..."}]}

### Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

1. Create a good prompt for a Frontier model - when we train our own models, we'll need to make the problem as easy as possible, but a Frontier model needs no such simplification.

> [{'role': 'system',
>
> 'content': 'You estimate prices of items. Reply only with the price, no explanation'},
> {'role': 'user',
> 'content': 'How much does this cost?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'},
> {'role': 'assistant', 'content': 'Price is $226.95'}]

In [ ]:
# First let's work on a good prompt for a Frontier model
# Notice that I'm removing the " to the nearest dollar"
# When we train our own models, we'll need to make the problem as easy as possible, 
# but a Frontier model needs no such simplification.

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

In [ ]:
messages_for(train[0])

2. Convert the items into a list of json objects - a "jsonl" string. Each row represents a message in the form: `{"messages" : [{"role": "system", "content": "You estimate prices...`

> {"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7"}, {"role": "assistant", "content": "Price is $226.95"}]}<br>
> {"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nPower Stop Rear Z36 Truck and Tow Brake Kit with Calipers\nThe Power Stop Z36 Truck & Tow Performance brake kit provides the superior stopping power demanded by those who tow boats, haul loads, tackle mountains, lift trucks, and play in the harshest conditions. The brake rotors are drilled to keep temperatures down during extreme braking and slotted to sweep away any debris for constant pad contact. Combined with our Z36 Carbon-Fiber Ceramic performance friction formulation, you can confidently push your rig to the limit and look good doing it with red powder brake calipers. Components are engineered to handle the stress of towing, hauling, mountainous driving, and lifted trucks. Dust-free braking performance. Z36 Carbon-Fiber Ceramic formula provides the extreme braking performance demanded by your truck or 4x"}, {"role": "assistant", "content": "Price is $506.98"}]}<br>
> {"messages": [{"role": "system", "content": "You estimate prices of items. Reply only with the price, no explanation"}, {"role": "user", "content": "How much does this cost?\n\nABBA 36 Gas Cooktop with 5 Sealed Burners - Tempered Glass Surface with SABAF Burners, Natural Gas Stove for Countertop, Home Improvement Essentials, Easy to Clean, 36 x 4.1 x 20.5\ncooktop Gas powered with 4 fast burners and 1 ultra-fast center burner Tempered glass surface with removable grid for easy cleaning Lightweight for easy installation. Installation Manual Included Counter cutout Dimensions 19 3/8 x 34 1/2 (see diagram) Insured shipping for your satisfaction and peace of mind Brand Name ABBA EST. 1956, Weight 30 pounds, Dimensions 20.5\\ D x 36\\ W x 4.1\\ H, Installation Type Count"}, {"role": "assistant", "content": "Price is $405.00"}]}

In [ ]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [ ]:
print(make_jsonl(train[:3]))

3. Convert the items into jsonl and write them to a file:

In [ ]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [ ]:
write_jsonl(fine_tune_train, "fine_tune_train.jsonl")

In [ ]:
write_jsonl(fine_tune_validation, "fine_tune_validation.jsonl")

In [ ]:
with open("fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [ ]:
train_file

4. Upload training and validation files to to OpenAI:

> FileObject(id='file-ANZ1fXNvT9rpFa5HptJ8fp', bytes=188742, created_at=1745807152, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

> FileObject(id='file-MGCRJXFiCjosPToMF5ywab', bytes=47085, created_at=1745807167, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

Note: rb means open it as a binary file to stream the files to OpenAI

In [ ]:
with open("fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [ ]:
validation_file

## Step 2: Fine-Tune OpenAI Model

To monitor fine-tuning register [Weights & Biases](https://wandb.ai) key on [OpenAI dashboard](https://platform.openai.com/account/organization) under Integrations on the General settings. Then create a wandb project.

Note: W&B service accounts (keys) are not available on the free tier.

Note: Validation file not necessary in this case but good to get in the habit of sending it

- With smaller training datasets usually multiple epochs are run but fixed to 1 below as a good sized trainign set used and can always run more training jobs with new training data. Remove it to let OpenAI decide how many epochs to use (number of traiing runs with data provided).
- Hyperparameters are what data scientists call the extra knobs and wheels and settings that control how your training is going to work. Any extra parameter you can set try different posibilities to see if it makes the model better or worse. This process of tryign our different values for better or worse is called _hyperparameter optimization_ or _hyperparameter tuning_, which is just fancy talk to _trial and error_.
- Remove integrations line if not monitoring with wandb
- Suffix is optinal too and will include this in the name of the model it creates

1. Execute fine tuning job:

> FineTuningJob(id='ftjob-PANNaovITUNfgU2fPnALCEDL', created_at=1745810086, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-UqfuFiZ3lRmy0bdF1SFE6SyU', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-ANZ1fXNvT9rpFa5HptJ8fp', validation_file='file-MGCRJXFiCjosPToMF5ywab', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-PANNaovITUNfgU2fPnALCEDL'))], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='pricer')

> SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-PANNaovITUNfgU2fPnALCEDL', created_at=1745810086, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-UqfuFiZ3lRmy0bdF1SFE6SyU', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-ANZ1fXNvT9rpFa5HptJ8fp', validation_file='file-MGCRJXFiCjosPToMF5ywab', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-PANNaovITUNfgU2fPnALCEDL'))], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1)), type='supervised'), user_provided_suffix='pricer')], has_more=False, object='list')

> 'ftjob-PANNaovITUNfgU2fPnALCEDL'

Start of job events:

> [FineTuningJobEvent(id='ftevent-qMjnvvRqSr0yRbcQngQTSJor', created_at=1745810086, level='info', message='Validating training file: file-ANZ1fXNvT9rpFa5HptJ8fp and validation file: file-MGCRJXFiCjosPToMF5ywab', object='fine_tuning.job.event', data={}, type='message'),
>
> > FineTuningJobEvent(id='ftevent-Qm93jkKFgINt9K1QhswVUU2U', created_at=1745810086, level='info', message='Created fine-tuning job: ftjob-PANNaovITUNfgU2fPnALCEDL', object='fine_tuning.job.event', data={}, type='message')]

In progess job events:

> [FineTuningJobEvent(id='ftevent-r70gslmVFgjy6rnD8uYnQnio', created_at=1745810442, level='info', message='Step 60/200: training loss=0.73, validation loss=1.46', object='fine_tuning.job.event', data={'step': 60, 'train_loss': 0.7305774688720703, 'valid_loss': 1.4598660469055176, 'total_steps': 200, 'train_mean_token_accuracy': 0.875, 'valid_mean_token_accuracy': 0.75}, type='metrics'),
> FineTuningJobEvent(id='ftevent-cLCUQkaOor0OjN2jYOPvVaO1', created_at=1745810440, level='info', message='Step 59/200: training loss=1.40', object='fine_tuning.job.event', data={'step': 59, 'train_loss': 1.401196002960205, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> FineTuningJobEvent(id='ftevent-5J9XiTALuGdFxCpc3Ss60DZ9', created_at=1745810437, level='info', message='Step 58/200: training loss=0.70', object='fine_tuning.job.event', data={'step': 58, 'train_loss': 0.7029500007629395, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> FineTuningJobEvent(id='ftevent-F2F9c4hUPA7E5c4lRHJXpz1W', created_at=1745810437, level='info', message='Step 57/200: training loss=1.14', object='fine_tuning.job.event', data={'step': 57, 'train_loss': 1.1434681415557861, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> FineTuningJobEvent(id='ftevent-bek67WqxTyneLmi0gzasG2NC', created_at=1745810437, level='info', message='Step 56/200: training loss=0.87', object='fine_tuning.job.event', data={'step': 56, 'train_loss': 0.8705952167510986, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> FineTuningJobEvent(id='ftevent-qEghJUp6zY22RAUoGpl3x4Yc', created_at=1745810434, level='info', message='Step 55/200: training loss=0.90', object='fine_tuning.job.event', data={'step': 55, 'train_loss': 0.9008762836456299, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> FineTuningJobEvent(id='ftevent-i8seByn9xcNDgQlMbpOZRaaj', created_at=1745810434, level='info', message='Step 54/200: training loss=1.44', object='fine_tuning.job.event', data={'step': 54, 'train_loss': 1.4444801807403564, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> FineTuningJobEvent(id='ftevent-G9onFQFBU6gukn1AAsxfSAdA', created_at=1745810434, level='info', message='Step 53/200: training loss=1.28', object='fine_tuning.job.event', data={'step': 53, 'train_loss': 1.2828950881958008, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> FineTuningJobEvent(id='ftevent-998fysaBBM2kG2rIDdVx7bes', created_at=1745810431, level='info', message='Step 52/200: training loss=0.63', object='fine_tuning.job.event', data={'step': 52, 'train_loss': 0.6335906982421875, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> FineTuningJobEvent(id='ftevent-medyvOgQsB7pNNMc1njFzB56', created_at=1745810431, level='info', message='Step 51/200: training loss=1.47', object='fine_tuning.job.event', data={'step': 51, 'train_loss': 1.4744696617126465, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics')

Completed job events:

> [FineTuningJobEvent(id='ftevent-bqb3dmLTCQ2ZCVaVbtATMcPJ', created_at=1745810672, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'),
>
> > FineTuningJobEvent(id='ftevent-ARwZfcP8WhglSCqT6vK5soEu', created_at=1745810668, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'),
> > FineTuningJobEvent(id='ftevent-RGffErhX5N3vxv4vkx9iGo6m', created_at=1745810630, level='info', message='Step 200/200: training loss=1.14, validation loss=1.13, full validation loss=1.12', object='fine_tuning.job.event', data={'step': 200, 'train_loss': 1.1376311779022217, 'valid_loss': 1.1295273303985596, 'total_steps': 200, 'full_valid_loss': 1.1193085956573485, 'train_mean_token_accuracy': 0.75, 'valid_mean_token_accuracy': 0.75, 'full_valid_mean_token_accuracy': 0.7925}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-SEA5eFcvrlKYbGA2O8XXNUiJ', created_at=1745810621, level='info', message='Step 199/200: training loss=1.42', object='fine_tuning.job.event', data={'step': 199, 'train_loss': 1.4206628799438477, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-6gCQzSFkCtI0pwZ5zRz0TTCY', created_at=1745810619, level='info', message='Step 198/200: training loss=0.52', object='fine_tuning.job.event', data={'step': 198, 'train_loss': 0.5175371170043945, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-53YUlCNBAVkBWHjo6RGdmmIu', created_at=1745810619, level='info', message='Step 197/200: training loss=1.24', object='fine_tuning.job.event', data={'step': 197, 'train_loss': 1.2442662715911865, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-tEVFXrITeqCn1wiCiFCtdhlY', created_at=1745810619, level='info', message='Step 196/200: training loss=0.87', object='fine_tuning.job.event', data={'step': 196, 'train_loss': 0.8681986331939697, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-pCTwhQtmrviENqKHuOwPFHmO', created_at=1745810616, level='info', message='Step 195/200: training loss=1.25', object='fine_tuning.job.event', data={'step': 195, 'train_loss': 1.2511980533599854, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-VKcs3M6Zyw1UXyooc9ZV3BQe', created_at=1745810616, level='info', message='Step 194/200: training loss=0.99', object='fine_tuning.job.event', data={'step': 194, 'train_loss': 0.9867780208587646, 'total_steps': 200, 'train_mean_token_accuracy': 0.875}, type='metrics'),
> > FineTuningJobEvent(id='ftevent-Xe26DwmMS5OJizJDt9BrdebI', created_at=1745810616, level='info', message='Step 193/200: training loss=1.23', object='fine_tuning.job.event', data={'step': 193, 'train_loss': 1.2297937870025635, 'total_steps': 200, 'train_mean_token_accuracy': 0.75}, type='metrics')]

<img src="./../images/Product-Pricer-Fine-Tuning-WandB-Job-Run.jpg" alt="OpenAI fine-tuning training job run monitored in Weigts and Bias" />

Better view of progress from wandb. What to look for:

- First few batch steps should show dramatic drop in train loss where model is leanring the structure and obvious stuff about the construct, like it is in dollars, prices and usually close to whole dollars, etc.
- What you are really looking for is continual progress from that point
- Good to see some variation, some batch steps with greater and lower loss, because your tryign to optimize and explore different possibilities
- Look for trend over time a gradual decrease in loss
- What could be a concern is if loss doesn't seem to be goign down
- Will pause at end of training to perform validation then send you an email whn the run completes

<img src="./../images/Product-Pricer-Fine-Tuning-WandB-Train-Loss-Graph.jpg" alt="OpenAI fine-tuning training train loss graph in Weights and Bias" />

Results:

- Good loss for most of the run
- Tappers off after 140 and gets a little worseafter 150(?!)

In [ ]:
wandb_integration = {"type": "wandb", "wandb": {"project": "gpt-pricer"}}

In [ ]:
train_file.id

In [ ]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    seed=42,
    hyperparameters={"n_epochs": 1},
#    integrations = [wandb_integration],
    suffix="pricer"
)

In [ ]:
openai.fine_tuning.jobs.list(limit=1)

In [ ]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [ ]:
job_id

In [ ]:
openai.fine_tuning.jobs.retrieve(job_id)

In [ ]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

## Step 3: Test our fine tuned model

Note: this can take a few hours to complete.

1. Get model name from job ID (or email from job complete notification):

> 'ft:gpt-4o-mini-2024-07-18:personal:pricer:BR9OWYkF'<br>
> 'ft:gpt-4o-mini-2024-07-18:personal:pricer:BmQm5j2P'<br>
> 'ft:gpt-4o-mini-2024-07-18:personal:pricer:BmQpBfzU'

In [ ]:
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [ ]:
fine_tuned_model_name

2. Update function to create prompt without price:

> [{'role': 'system',<br>
>   'content': 'You estimate prices of items. Reply only with the price, no explanation'},<br>
> {'role': 'user',<br>
>  'content': "How much does this cost?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"},<br>
> {'role': 'assistant', 'content': 'Price is $'}]

In [ ]:
# The prompt

def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [ ]:
# Try this out

messages_for(test[0])

3. Create a utility function to extract the price from a string:

In [ ]:
# A utility function to extract the price from a string

def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [ ]:
get_price("The price is roughly $99.99 because blah blah")

4. Create the function for our fine tuned gpt-4o-mini model:

> 374.41<br>
> print(gpt_fine_tuned(test[0]))<br>
> 174.77

In [ ]:
# The function for gpt-4o-mini

def gpt_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name, 
        messages=messages_for(item),
        seed=42,
        max_tokens=7
    )
    reply = response.choices[0].message.content
    return get_price(reply)

In [ ]:
print(test[0].price)
print(gpt_fine_tuned(test[0]))

In [ ]:
print(test[0].test_prompt())

5. Test the model against all the test data:

In [ ]:
Tester.test(gpt_fine_tuned, test)